In [1]:
# syft absolute
import syft as sy
from syft import UID

In [2]:
# we can generate a thin wrapper like action object but without the bloat
# we synthesize a custom type so that it acts like the real type for 99.9999% of cases

In [3]:
# stdlib
from typing import Any
from typing import Generic
from typing import TypeVar

T = TypeVar("T")


class SyftAutoboxMeta(type):
    def __call__(cls, value: T):
        wrapped_type = type(value)
        dynamic_class_name = f"SyftAutobox[{wrapped_type.__name__}]"

        # Dynamically create a new class that inherits from the wrapped type only
        DynamicWrapper = type(
            f"SyftAutobox{wrapped_type.__name__.capitalize()}", (wrapped_type,), {}
        )

        class Wrapped(DynamicWrapper):
            def __init__(self, value: Any, uid: UID | None = None):
                self._syft_value = value
                self._syft_uid = uid if uid is not None else UID()
                super(DynamicWrapper, self).__init__()

            def __getattribute__(self, name: str) -> Any:
                # Bypass certain attrs to prevent recursion issues
                if name.startswith("_syft"):
                    return object.__getattribute__(self, name)

                return getattr(self._syft_value, name)

            # these empty dunders are required
            def __repr__(self):
                return self.__repr__()

            def __str__(self):
                return self.__str__()

            def __dir__(self):
                return self.__dir__()

        Wrapped.__name__ = dynamic_class_name
        Wrapped.__qualname__ = dynamic_class_name
        return Wrapped(value)


class SyftAutobox(Generic[T], metaclass=SyftAutoboxMeta):
    pass

In [4]:
# a vanilla string
b = "test"

In [5]:
# a syft autoboxed string
a = SyftAutobox(value=b)

In [6]:
a

'test'

In [7]:
# type is nice and clean
type(a), type(b)

(__main__.SyftAutobox[str], str)

In [8]:
# passes isinstance check
assert isinstance(a, str) and isinstance(b, str)

In [9]:
# passes is subclass check
assert issubclass(type(a), str) and issubclass(type(b), str)

In [10]:
# we renamed one class so it looks nested
type(a).mro()

[__main__.SyftAutobox[str], __main__.SyftAutoboxStr, str, object]

In [11]:
type(b).mro()

[str, object]

In [12]:
# lets try numpy

In [13]:
# third party
import numpy as np

# Create a numpy array
arr = np.array([1, 2, 3, 4, 5])

# Define a function that expects a numpy array


def sum_array(arr: np.ndarray) -> int:
    return np.sum(arr)


# Test with a regular numpy array
print(sum_array(arr))  # Should print 15

15


In [14]:
arr

array([1, 2, 3, 4, 5])

In [15]:
# Wrap the numpy array
wrapped_arr = SyftAutobox(arr)

In [16]:
wrapped_arr

array([1, 2, 3, 4, 5])

In [17]:
sum_array(wrapped_arr)

15

In [18]:
# nice
wrapped_arr._syft_uid

<UID: f1bed246b5314d97b689ae76beacc037>

In [19]:
# third party
# lets try some stuff which might break with ActionObject
import numpy.ctypeslib as npct


def to_ctypes(arr: np.ndarray):
    return npct.as_ctypes(arr)

In [20]:
action_arr = sy.ActionObject.from_obj(arr)
action_arr


**Pointer**

array([1, 2, 3, 4, 5])


In [21]:
try:
    print(to_ctypes(action_arr))  # This will raise an exception
except TypeError as e:
    print(f"TypeError: {e}")

TypeError: Cannot interpret 'Pointer:
'<i8'' as a data type


In [22]:
# works for raw numpy
try:
    print(to_ctypes(arr))  # This will raise an exception
except TypeError as e:
    print(f"TypeError: {e}")

In [23]:
# works for autobox
try:
    print(to_ctypes(wrapped_arr))  # This will raise an exception
except TypeError as e:
    print(f"TypeError: {e}")

In [24]:
# okay now what if we made this wrapper type capable of intelligently handling Results?

In [448]:
# stdlib
from typing import Generic
from typing import TypeVar

# syft absolute
from syft import ActionObject

T = TypeVar("T")
E = TypeVar("E")


class Ok(Generic[T]):
    def __init__(self, value: T):
        self.value = value


class Err(Generic[E]):
    def __init__(self, error: E):
        self.error = error


Result = Ok[T] | Err[E]

error_allowed_attrs = ["__class__"]
skip_attrs = ["_internal_names_set"]


class SyftAutoboxMeta(type):
    def __call__(cls, value: Result[T, E] | T):
        if isinstance(value, Ok):
            wrapped_value = value.value
            is_error = False
        elif isinstance(value, Err):
            wrapped_value = value.error
            is_error = True
        else:
            wrapped_value = value
            is_error = False

        uid = None
        if isinstance(value, ActionObject):
            wrapped_value = value.syft_action_data
            uid = value.id

        wrapped_type = type(wrapped_value)
        dynamic_class_name = f"SyftAutobox[{wrapped_type.__name__}]"

        # Dynamically create a new class that inherits from the wrapped type only
        DynamicWrapper = type(
            f"SyftAutobox{wrapped_type.__name__.capitalize()}", (wrapped_type,), {}
        )

        class Wrapped(DynamicWrapper):
            def __init__(self, value: Any, uid: UID | None = None):
                self._syft_value = value
                self._syft_uid = uid if uid is not None else UID()
                self._syft_is_error = is_error
                super(DynamicWrapper, self).__init__()

            def __getattribute__(self, name: str) -> Any:
                # Bypass certain attrs to prevent recursion issues
                if name.startswith("_syft") or name in skip_attrs:
                    return object.__getattribute__(self, name)

                if self._syft_is_error and name not in error_allowed_attrs:
                    raise Exception(
                        f"Cannot access attribute '{name}' on an Err result: {self._syft_value}"
                    )

                return getattr(self._syft_value, name)

            # def __getattr__(self, name: str) -> Any:
            #     if name.startswith("_syft") or name in skip_attrs:
            #         return getattr(self, name)
            #     return getattr(self._syft_value, name)

            def __setattr__(self, name, value) -> None:
                if name.startswith("_syft"):
                    object.__setattr__(self, name, value)
                    return
                setattr(self._syft_value, name, value)

            # these empty dunders are required
            def __repr__(self):
                return self.__repr__()

            def __str__(self):
                return self.__str__()

            def __dir__(self):
                return self.__dir__()

            def __getitem__(self, name):
                return self.__getitem__(name)

            def __setitem__(self, name, value):
                return self.__setitem__(name, value)

            def __array__(self):
                return self.__array__()

            def __len__(self):
                return len(self._syft_value)

            def __iter__(self):
                return iter(self._syft_value)

            def __contains__(self, item):
                return item in self._syft_value

            def __call__(self, *args, **kwargs):
                return self(*args, **kwargs)

            def __eq__(self, other):
                return self == other

            def __ne__(self, other):
                return self != other

            def __lt__(self, other):
                return self < other

            def __le__(self, other):
                return self <= other

            def __gt__(self, other):
                return self > other

            def __ge__(self, other):
                return self >= other

            def __add__(self, other):
                return self + other

            def __sub__(self, other):
                return self - other

            def __mul__(self, other):
                return self * other

            def __truediv__(self, other):
                return self / other

            def __floordiv__(self, other):
                return self // other

            def __mod__(self, other):
                return self % other

            def __pow__(self, other, modulo=None):
                return pow(self._syft_value, other, modulo)

            def __radd__(self, other):
                return other + self._syft_value

            def __rsub__(self, other):
                return other - self._syft_value

            def __rmul__(self, other):
                return other * self._syft_value

            def __rtruediv__(self, other):
                return other / self._syft_value

            def __rfloordiv__(self, other):
                return other // self._syft_value

            def __rmod__(self, other):
                return other % self._syft_value

            def __rpow__(self, other):
                return pow(other, self._syft_value)

            def __neg__(self):
                return -self._syft_value

            def __pos__(self):
                return +self._syft_value

            def __abs__(self):
                return abs(self._syft_value)

            def __invert__(self):
                return ~self._syft_value

            def __and__(self, other):
                return self & other

            def __or__(self, other):
                return self | other

            def __xor__(self, other):
                return self ^ other

            def __lshift__(self, other):
                return self << other

            def __rshift__(self, other):
                return self >> other

            def __rand__(self, other):
                return other & self

            def __ror__(self, other):
                return other | self

            def __rxor__(self, other):
                return other ^ self

            def __rlshift__(self, other):
                return other << self

            def __rrshift__(self, other):
                return other >> self

        Wrapped.__name__ = dynamic_class_name
        Wrapped.__qualname__ = dynamic_class_name
        obj = Wrapped(wrapped_value)
        if uid:
            obj._syft_uid = uid
        return obj


class SyftAutobox(Generic[T], metaclass=SyftAutoboxMeta):
    pass

In [449]:
# now we can freely return T or Result[T] and its the same
ok_value = SyftAutobox(Ok(42))
print(ok_value)  # Should print 42

42


In [450]:
type(ok_value)

__main__.SyftAutobox[int]

In [451]:
isinstance(ok_value, int)

True

In [452]:
# can perform addition, and can probably autowrap if we go and implement the __ like in actionobj
b = ok_value + 1
type(b)

RecursionError: maximum recursion depth exceeded

In [ ]:
# what about an error

In [ ]:
# s = sy.SyftError(message="Some Error")

In [ ]:
try:
    err_value = SyftAutobox(Err("Some Error"))
    print(err_value.upper())  # This should raise an exception
except Exception as e:
    print(e)  # Should raise an exception with the error message

In [453]:
# even though its got a string inside it should raise exceptions
err_value.upper()

Exception: Cannot access attribute 'upper' on an Err result: [1 2 3 4 5]

In [454]:
# we can choose to raise exceptions when accessing error types however we like

In [455]:
try:
    err_value = SyftAutobox(Err(arr))
    print(err_value.dtype)  # This should raise an exception
except Exception as e:
    print(e)  # Should raise an exception with the error message

Cannot access attribute 'dtype' on an Err result: [1 2 3 4 5]


In [456]:
# rationale
# better than ActionObject already, as it passes more C code checks etc
# allows us to add some Result / Future functionality ONTOP of this type as desired
# allows a UID for linkage at all times
# solves issue of returning a Result[T, E] vs T raises Exception theres no difference when you autobox
# raises exceptions if we want it to

In [457]:
# TODO:
# autowrap results of infix operators
# raise exceptions on infix operators if its an Exception type
# make exception raising optional
# add is_ok, is_err, type methods to the Autobox
# add ._syft_ready, .is_ready
# Future
# __getattr__ -> self._syft_value
# check if its been longer than 1 second
# if name of getattr not in special ._syft_ready, .is_ready
# call server for update and then replace self._syft_value with real value

# obj.sum() <---

In [458]:
# method -> T, Exception
# method_async -> Result[T, Error]

In [459]:
# method -> Autobox[T, SyftErr[Message, Exception]]

# TBD
# method_async -> Autobox[Future[T, Exception], SyftErr[Message, Exception]]

In [460]:
# third party
import pandas as pd

In [461]:
data = {
    "Name": ["Alice", "Bob", "Charlie"],
    "Age": [25, 30, 35],
    "City": ["New York", "Los Angeles", "Chicago"],
}

In [462]:
df = pd.DataFrame(data)
df

,Name,Age,City
0,Alice,25,New York
1,Bob,30,Los Angeles
2,Charlie,35,Chicago


In [463]:
d = sy.ActionObject.from_obj(data)
d


**Pointer**

{'Name': ['Alice', 'Bob', 'Charlie'], 'Age': [25, 30, 35], 'City': ['New York', 'Los Angeles', 'Chicago']}


In [464]:
df2 = pd.DataFrame(d)
df2

,0,1,2,3
0,(((N))),(((a))),(((m))),(((e)))
1,(((A))),(((g))),(((e))),None
2,(((C))),(((i))),(((t))),(((y)))


In [465]:
# stdlib

In [466]:
# class PandasSyftAutobox(SyftAutobox):
#     def __dataframe__(self, *args: Any, **kwargs: Any) -> Any:
#         return self.__dataframe__(*args, **kwargs)

#     def __getattribute__(self, name: str) -> Any:
#         if self._syft_is_property(self, name):
#             return getattr(self._syft_value, name)
#         return super().__getattribute__(name)

#     def _syft_get_property(self, obj: Any, method: str) -> Any:
#         return getattr(self._syft_value, method)

#     def _syft_is_property(self, obj: Any, method: str) -> bool:
#         cols = self._syft_value.columns.values.tolist()
#         if method in cols:
#             return True

#         klass_method = getattr(type(obj), method, None)
#         return isinstance(klass_method, property) or inspect.isdatadescriptor(
#             klass_method
#         )

#     def __bool__(self) -> bool:
#         return bool(self._syft_value)

In [467]:
df = pd.DataFrame(data)
df

,Name,Age,City
0,Alice,25,New York
1,Bob,30,Los Angeles
2,Charlie,35,Chicago


In [468]:
auto_dict = SyftAutobox(data)

In [469]:
type(auto_dict)

__main__.SyftAutobox[dict]

In [470]:
auto_dict

{'Name': ['Alice', 'Bob', 'Charlie'],
 'Age': [25, 30, 35],
 'City': ['New York', 'Los Angeles', 'Chicago']}

In [471]:
df3 = pd.DataFrame(auto_dict)
df3

,Name,Age,City
0,Alice,25,New York
1,Bob,30,Los Angeles
2,Charlie,35,Chicago


In [484]:
df_ao = sy.ActionObject.from_obj(df3)
print(df_ao.id)
df_ao

4c525e129ee84e4c8e9abb3c4ecd95fd


,Name,Age,City
0,Alice,25,New York
1,Bob,30,Los Angeles
2,Charlie,35,Chicago


In [485]:
# action object fails

In [486]:
# autobox to the rescue
df_ob = SyftAutobox(df_ao)
print(df_ob._syft_uid)
df_ob

4c525e129ee84e4c8e9abb3c4ecd95fd


,Name,Age,City
0,Alice,25,New York
1,Bob,30,Los Angeles
2,Charlie,35,Chicago


In [487]:
pd.DataFrame(df_ob)

,Name,Age,City
0,Alice,25,New York
1,Bob,30,Los Angeles
2,Charlie,35,Chicago


In [474]:
pd.DataFrame(df_ao)

ValueError: object __array__ method not producing an array

In [475]:
df_ob = SyftAutobox(df3)
df_ob

,Name,Age,City
0,Alice,25,New York
1,Bob,30,Los Angeles
2,Charlie,35,Chicago


In [490]:
df.Age.sum()

90

In [495]:
assert all(df == df)

In [488]:
df.sum("Age")

ValueError: No axis named Age for object type DataFrame

In [476]:
type(df_ob)

__main__.SyftAutobox[DataFrame]

In [477]:
pd.DataFrame(df_ob)

,Name,Age,City
0,Alice,25,New York
1,Bob,30,Los Angeles
2,Charlie,35,Chicago


In [478]:
np.array(df_ob)

array([['Alice', 25, 'New York'],
       ['Bob', 30, 'Los Angeles'],
       ['Charlie', 35, 'Chicago']], dtype=object)